# GEEMAP ONLINE AND OFFLINE DATA ACCESS 
## (PART 2 - MULTIBAND SPATIAL DATA)

Course: CEE5003 Remote Sensing of Land Surfaces  

Created by: Alfonso Torres  

Date: May 2024

***

### Goal 

Demonstrate the different data access and retrieval when using the Google Earth Engine 
***

### Data Sources

* Cloud:  
 a. GEE [NLDAS-2: North American Land Data Assimilation System Forcing Fields](https://developers.google.com/earth-engine/datasets/catalog/NASA_NLDAS_FORA0125_H002#description)    

   Products of interest

   precipitation "total_precipitation"	  
   air temperature at 2 meters "temperature"  
   evapotranspiration "potential_evaporation"

 b. GEE [TIGER: US Census States 2018](https://developers.google.com/earth-engine/datasets/catalog/TIGER_2018_States)  




* Local:
    None  

***

### Implementation

1. Importing modules

In [98]:
import ee  # Import the Earth Engine Python API
import geemap  # Import the geemap library for interactive mapping
import eemont  # Import the eemont library for extending the capabilities of the Earth Engine Python API

# Import the pprint module for presenting metadata information from rasters/images from Google Earth Engine
from pprint import pprint

In [99]:
##### uncomment this cell and run it if the cell below fails
# import geemap
# m = geemap.Map()
# m

2. Initialize Earth Engine

In [101]:
ee.Initialize()

### Accessing Different Multiband Image Products

3. Load Utah Boundary

In [104]:
# Load the 'TIGER/2018/Counties' FeatureCollection and filter it to get the feature for 'Cache' county
boundary_cache_county = (
    ee.FeatureCollection("TIGER/2018/Counties")
    .filter(ee.Filter.eq('NAME', 'Cache'))
    .first()
)

# Print the information of the 'boundary_cache_county' feature
# pprint(boundary_cache_county.getInfo())

Loading NAIP imagery

NAIP imagery have minimal cloud occurrence

In [106]:
# Load the NAIP dataset within the specified date range
dataset = (
    ee.ImageCollection("USDA/NAIP/DOQQ")
    .filterDate('2021-01-01' ,'2024-12-31')  # Filter images within the date range
    .mosaic()  # Create a single mosaic image from the collection
)

# Print information about the dataset
pprint(dataset.getInfo())

{'bands': [{'crs': 'EPSG:4326',
            'crs_transform': [1, 0, 0, 0, 1, 0],
            'data_type': {'max': 255,
                          'min': 0,
                          'precision': 'int',
                          'type': 'PixelType'},
            'id': 'R'},
           {'crs': 'EPSG:4326',
            'crs_transform': [1, 0, 0, 0, 1, 0],
            'data_type': {'max': 255,
                          'min': 0,
                          'precision': 'int',
                          'type': 'PixelType'},
            'id': 'G'},
           {'crs': 'EPSG:4326',
            'crs_transform': [1, 0, 0, 0, 1, 0],
            'data_type': {'max': 255,
                          'min': 0,
                          'precision': 'int',
                          'type': 'PixelType'},
            'id': 'B'},
           {'crs': 'EPSG:4326',
            'crs_transform': [1, 0, 0, 0, 1, 0],
            'data_type': {'max': 255,
                          'min': 0,
                          

In [107]:
dataset = dataset.clip(boundary_cache_county)

In [108]:
naturalColor = dataset.select(['R', 'G', 'B']);

naturalColorVis = {
    'min': 0, #min value to force image to display
    'max': 255, #max value to fore image to display
    'gamma': [1.2, 1.1, 1], #color enhancement (to highlight low/medium values)
}

Map1 = geemap.Map()
Map1.centerObject(boundary_cache_county, 10)

Map1.addLayer(naturalColor, naturalColorVis, '2021 USDA NAIP');

Map1

Map(center=[41.72212265509924, -111.74343433294621], controls=(WidgetControl(options=['position', 'transparent…

In [109]:
falseColor = dataset.select(['N', 'R', 'G']);

falseColorVis = {
    'min': 0, #min value to force image to display
    'max': 255, #max value to fore image to display
    'gamma': [1.2, 1, 1], #color enhancement (to highlight low/medium values)
}

Map2 = geemap.Map()
Map2.centerObject(boundary_cache_county, 10)

Map2.addLayer(falseColor, falseColorVis, '2021 false color USDA NAIP');

Map2

Map(center=[41.72212265509924, -111.74343433294621], controls=(WidgetControl(options=['position', 'transparent…

Loading Sentinel2 Imagery

Because this imagery is on schedule (every 5 days, clouds and shadows are bound to happen. We need to build a cloud filter.

In [111]:
def mask_s2_clouds(sentinel2_image):
  """Masks clouds in a Sentinel-2 image using the QA band.

  Args:
      sentinel2_image (ee.Image): A Sentinel-2 image.

  Returns:
      ee.Image: A cloud-masked Sentinel-2 image.
  """
  qa = sentinel2_image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloud_bit_mask = 1 << 10
  cirrus_bit_mask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = (
      qa.bitwiseAnd(cloud_bit_mask)
      .eq(0)
      .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
  )

  return sentinel2_image.updateMask(mask).divide(10000)

In [112]:
dataset = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterDate('2023-06-01', '2023-06-30')
#     .filterBounds(boundary_cache_county.geometry())
    # Pre-filter to get less cloudy granules.
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))
    .map(mask_s2_clouds)
    .mosaic() # to get a single image change this command to first(), for mosaic, use mosaic()
)


dataset = dataset.clip(boundary_cache_county)

visualization = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2'],
}

Map3 = geemap.Map()
Map3.centerObject(boundary_cache_county, 10)
Map3.add_layer(dataset, visualization, 'RGB')
Map3

Map(center=[41.72212265509924, -111.74343433294621], controls=(WidgetControl(options=['position', 'transparent…

Accesing Sentinel 3 imagery

In [114]:
dataset = (
    ee.ImageCollection('COPERNICUS/S3/OLCI')
#     .filterDate('2024-05-29', '2024-05-30')
    .filterDate('2024-06-01', '2024-06-30')
#     .filterBounds(boundary_cache_county.geometry())
    .mosaic() # to get a single image change this command to first(), for mosaic, use mosaic()
)

# Select bands for visualization and apply band-specific scale factors.
rgb = (
    dataset.select(['Oa08_radiance', 'Oa06_radiance', 'Oa04_radiance'])
    #Convert to radiance units.
    .multiply(ee.Image([0.00876539, 0.0123538, 0.0115198]))
)

rgb = rgb.clip(boundary_cache_county)


visParams = {
  'min': 0,
  'max': 3,
  'gamma': 1.5,
}

Map4 = geemap.Map()
Map4.centerObject(boundary_cache_county, 10)
Map4.addLayer(rgb, visParams, 'RGB')
Map4

Map(center=[41.72212265509924, -111.74343433294621], controls=(WidgetControl(options=['position', 'transparent…

Accesing MODIS Terra

In [116]:
dataset = (
    ee.ImageCollection('MODIS/061/MOD09GA')
    .filterDate('2024-04-01', '2024-05-30')
    .maskClouds()
#     .filterBounds(boundary_cache_county.geometry())
    .mosaic() # to get a single image change this command to first(), for mosaic, use mosaic()
)


trueColor143 = (
    dataset.select(['sur_refl_b01', 'sur_refl_b04', 'sur_refl_b03']).divide(10000)
)

trueColor143 = trueColor143.clip(boundary_cache_county)

    
trueColor143Vis = {
  'min': 0,
  'max': 0.3,
}

Map5 = geemap.Map()
Map5.centerObject(boundary_cache_county, 10)
Map5.addLayer(trueColor143, trueColor143Vis, 'True Color (143)')
Map5

Map(center=[41.72212265509924, -111.74343433294621], controls=(WidgetControl(options=['position', 'transparent…

Accesing Landsat imagery

In [118]:
# Applies scaling factors to the Landsat image bands.
def apply_scale_factors(landsat_image):
  # Scale the optical bands.
  optical_bands = landsat_image.select('SR_B.').multiply(0.0000275).add(-0.2)
  # Scale the thermal bands.
  thermal_bands = landsat_image.select('ST_B.*').multiply(0.00341802).add(149.0)
  # Add the scaled bands to the Landsat image.
  return landsat_image.addBands(optical_bands, None, True).addBands(
      thermal_bands, None, True
  )

# Load the Landsat image collection and apply scaling factors.
dataset = (
    ee.ImageCollection('LANDSAT/LC09/C02/T1_L2')
    .filterDate('2024-04-01', '2024-05-30')
    .maskClouds()
    .map(apply_scale_factors)
    .mosaic()  # To get a single image, change this command to first(). For mosaic, use mosaic().
)

# Clip the dataset to the specified boundary.
dataset = dataset.clip(boundary_cache_county)

# Define visualization parameters.
visualization = {
    'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
    'min': 0.0,
    'max': 0.5,
}

# Create a map and add the dataset with the specified visualization parameters.
Map6 = geemap.Map()
Map6.centerObject(boundary_cache_county, 10)
Map6.add_layer(dataset, visualization, 'True Color (432)')
Map6

Map(center=[41.72212265509924, -111.74343433294621], controls=(WidgetControl(options=['position', 'transparent…